In [1]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from scipy.optimize import minimize

from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Загружаем данные
для каждого phone_id показаны стоимости сделанных заказов в месяц (с января по август) 

In [2]:
data = pd.read_csv('/content/drive/My Drive/данные/train_try.csv')
data.drop(columns=['Unnamed: 0'], inplace=True)
data.head()

,phone_id,price1,price2,price3,price4,price5,price6,price7,price8
0,1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,2,3827.990007,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,2,2902.410011,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,2,0.000000,2622.290012,0.000000,0.0,0.0,0.0,0.0,0.0
4,2,0.000000,0.000000,4716.760031,0.0,0.0,0.0,0.0,0.0


Просуммируем для каждого phone_id общую сумму заказов в месяц (с января по август)

In [3]:
data_fr = data.rename(columns={'price1': '2020-01', 'price2': '2020-02','price3': '2020-03','price4': '2020-04',
                        'price5': '2020-05','price6': '2020-06','price7': '2020-07','price8': '2020-08' })
X = data_fr[['2020-01', '2020-02','2020-03','2020-04', '2020-05','2020-06','2020-07','2020-08' ,'phone_id']].groupby(['phone_id']).sum()
X


,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08
phone_id,,,,,,,,
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,6730.400019,2622.290012,4716.760031,0.000000,5684.450022,0.000000,0.000000,0.000000
3,0.000000,0.000000,6324.900032,0.000000,3919.770002,0.000000,1362.600008,0.000000
4,0.000000,0.000000,10110.380024,14323.269983,8546.550030,5622.109989,0.000000,0.000000
5,0.000000,0.000000,1169.480034,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
719370,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2417.999990
719371,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
719372,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Так как phone_id слишком много, то будем считать по группам (10 000 за раз).  Для этого сделаем цикл, с помощью которого посчитаем для 710 000 phone_id (далее переименуем в Id). Результаты предсказания модели записываются в датафрейм y_pred_2 

In [4]:
from statsmodels.tsa.vector_ar.var_model import VAR

n_1 = 0
n_2 = 10000
res = np.array([[], []])
for i in range(1,72):

  X_10 = X.iloc[n_1:n_2]
  X_cat = X_10.T
  model = VAR(X_cat)
  model_fit = model.fit()
  
  yhat = model_fit.forecast(model_fit.y, steps=1)
  X_ind = np.array(X.reset_index().phone_id[n_1:n_2])

  pred = np.vstack([[X_ind], yhat])

  n_1 = n_2
  n_2 = n_2 + 10000
  res = np.hstack((res, pred))

  del pred

pred_2 = pd.DataFrame(data = res)
y_pred_2 = pred_2.T
y_pred_2 = y_pred_2.rename(columns={0:'Id', 1:'predicted'})

Теперь посчитаем предсказания модели для оставшихся Id в y_pred3

In [5]:
n_1 = 710000
n_2 = 719374

X_10 = X.iloc[n_1:n_2]
X_cat = X_10.T
model = VAR(X_cat)
model_fit = model.fit()

yhat3 = model_fit.forecast(model_fit.y, steps=1)
print(yhat3)

res3 = np.array(X.reset_index().phone_id[n_1:n_2])
res3 = np.vstack([[res3], yhat3])

pred3 = pd.DataFrame(data = res3)
y_pred3 = pred3.T
y_pred3 = y_pred3.rename(columns={0:'Id',1:'predicted'})


[[ 472.22019988 1430.76655494  727.18427096 ...    0.
     3.32633625    1.51778763]]


Объединим полученные предсказания в общую таблицу и проанализируем стат. параметры 

In [6]:
all_w = pd.concat([y_pred_2, y_pred3])
all_w.describe()


,Id,predicted
count,719374.000000,719374.000000
mean,359687.500000,1260.637566
std,207665.530611,3288.233366
min,1.000000,-30217.988335
25%,179844.250000,0.071503
50%,359687.500000,196.033239
75%,539530.750000,1140.800710
max,719374.000000,568702.798994


Создадим новый столбец по условию: если предсказанная сумма покупок в месяц больше 160 -> 1 (предполагаем, что клиент не уйтет), в противном случае -> 0

In [7]:
all_w['Predicted'] = np.where(all_w['predicted'] > 160, 1, 0)

Изменим тип в int32

In [8]:
all_w = all_w.astype('int32')
all_w.dtypes

Id           int32
predicted    int32
Predicted    int32
dtype: object

Предсказать отток требуется только для определленых значений Id, которые указаны в файле sample_submission.csv

In [11]:
data_omg = pd.read_csv('/content/sample_submission.csv' , sep=';')
data_omg = data_omg.drop(columns=['Predicted'])

После того как мы считали из файла Id, выполним left join с таблицей all_w, чтобы узнать предсказания только для указанных Id из файла

In [14]:
result = pd.merge(data_omg, all_w, on='Id', how='left')
result

,Id,predicted,Predicted
0,19843,2419,1
1,471287,344,1
2,342522,197,1
3,457799,1727,1
4,233778,2864,1
...,...,...,...
214604,16841,9371,1
214605,69348,952,1
214606,594974,1620,1
214607,320024,1093,1


Сохраним результат в таблицу sample_sub 

In [13]:
result = result.astype('int32')
result.to_csv('sample_sub.csv', columns=['Id', 'Predicted'], index=None)